In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv("../../data/ner_dataset.csv", encoding="latin1")

In [3]:
data = data.fillna(method="ffill")

In [4]:
data.tail(10)

,Sentence #,Word,POS,Tag
1048565,Sentence: 47958,impact,NN,O
1048566,Sentence: 47958,.,.,O
1048567,Sentence: 47959,Indian,JJ,B-gpe
1048568,Sentence: 47959,forces,NNS,O
1048569,Sentence: 47959,said,VBD,O
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [5]:
words = list(set(data["Word"].values))

In [6]:
n_words = len(words); n_words

35178

In [7]:
class SentenceGetter(object):
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [8]:
getter = SentenceGetter(data)

In [9]:
sent = getter.get_next()

In [10]:
print(sent)

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


In [12]:
# 모든 문장
sentences = getter.sentences

In [14]:
sentences[0]

[('Thousands', 'NNS', 'O'),
 ('of', 'IN', 'O'),
 ('demonstrators', 'NNS', 'O'),
 ('have', 'VBP', 'O'),
 ('marched', 'VBN', 'O'),
 ('through', 'IN', 'O'),
 ('London', 'NNP', 'B-geo'),
 ('to', 'TO', 'O'),
 ('protest', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('war', 'NN', 'O'),
 ('in', 'IN', 'O'),
 ('Iraq', 'NNP', 'B-geo'),
 ('and', 'CC', 'O'),
 ('demand', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('withdrawal', 'NN', 'O'),
 ('of', 'IN', 'O'),
 ('British', 'JJ', 'B-gpe'),
 ('troops', 'NNS', 'O'),
 ('from', 'IN', 'O'),
 ('that', 'DT', 'O'),
 ('country', 'NN', 'O'),
 ('.', '.', 'O')]

## Feature preparation 
- Craft a set of features and prepare the dataset

In [23]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()':word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
        
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]


def sent2labels(sent):
    return [label for token, postag, label in sent]


def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [24]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [25]:
print(len(X), len(y))

47959 47959


In [26]:
sentences[0]

[('Thousands', 'NNS', 'O'),
 ('of', 'IN', 'O'),
 ('demonstrators', 'NNS', 'O'),
 ('have', 'VBP', 'O'),
 ('marched', 'VBN', 'O'),
 ('through', 'IN', 'O'),
 ('London', 'NNP', 'B-geo'),
 ('to', 'TO', 'O'),
 ('protest', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('war', 'NN', 'O'),
 ('in', 'IN', 'O'),
 ('Iraq', 'NNP', 'B-geo'),
 ('and', 'CC', 'O'),
 ('demand', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('withdrawal', 'NN', 'O'),
 ('of', 'IN', 'O'),
 ('British', 'JJ', 'B-gpe'),
 ('troops', 'NNS', 'O'),
 ('from', 'IN', 'O'),
 ('that', 'DT', 'O'),
 ('country', 'NN', 'O'),
 ('.', '.', 'O')]

In [28]:
X[0][1]

{'bias': 1.0,
 'word.lower()': 'of',
 'word[-3:]': 'of',
 'word[-2]': 'of',
 'word.isupper()': False,
 'word.istitle()': False,
 'word.isdigit()': False,
 'postag': 'IN',
 'postag[:2]': 'IN',
 '-1:word.lower()': 'thousands',
 '-1:word.istitle()': True,
 '-1:word.isupper()': False,
 '-1:postag': 'NNS',
 '-1:postag[:2]': 'NN',
 '+1:word.lower()': 'demonstrators',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'NNS',
 '+1:postag[:2]': 'NN'}

## Fit the CRF model 

In [30]:
# !pip install sklearn-crfsuite

In [31]:
from sklearn_crfsuite import CRF

crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.2,
          max_iterations=100,
          all_possible_transitions=False)

In [32]:
from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report

In [33]:
%%time
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

/home1/irteam/anaconda3/envs/py37_tf20/lib/python3.7/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CPU times: user 20min 9s, sys: 6.33 s, total: 20min 15s
Wall time: 20min 15s


## Evaluate the model

In [35]:
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

              precision    recall  f1-score   support

       B-art       0.38      0.10      0.15       402
       B-eve       0.53      0.35      0.42       308
       B-geo       0.85      0.91      0.88     37644
       B-gpe       0.97      0.94      0.95     15870
       B-nat       0.67      0.37      0.47       201
       B-org       0.79      0.72      0.75     20143
       B-per       0.83      0.82      0.83     16990
       B-tim       0.93      0.88      0.90     20333
       I-art       0.12      0.03      0.04       297
       I-eve       0.36      0.22      0.27       253
       I-geo       0.82      0.80      0.81      7414
       I-gpe       0.92      0.55      0.69       198
       I-nat       0.54      0.27      0.36        51
       I-org       0.81      0.79      0.80     16784
       I-per       0.84      0.90      0.87     17251
       I-tim       0.84      0.76      0.80      6528
           O       0.99      0.99      0.99    887908

    accuracy              

In [36]:
%%time
crf.fit(X, y)

CPU times: user 4min 58s, sys: 1.1 s, total: 4min 59s
Wall time: 4min 59s


/home1/irteam/anaconda3/envs/py37_tf20/lib/python3.7/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.2, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [37]:
!pip install eli5

     |████████████████████████████████| 112kB 965kB/s eta 0:00:01


In [38]:
import eli5

/home1/irteam/anaconda3/envs/py37_tf20/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home1/irteam/anaconda3/envs/py37_tf20/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [39]:
eli5.show_weights(crf, top=30)

/home1/irteam/anaconda3/envs/py37_tf20/lib/python3.7/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


From \ To,O,B-art,I-art,B-eve,I-eve,B-geo,I-geo,B-gpe,I-gpe,B-nat,I-nat,B-org,I-org,B-per,I-per,B-tim,I-tim
O,4.315,0.91,0.0,1.906,0.0,2.06,0.0,1.414,0.0,1.646,0.0,2.176,0.0,4.351,0.0,2.925,0.0
B-art,-0.185,0.0,7.402,0.0,0.0,-0.518,0.0,0.0,0.0,0.0,0.0,0.509,0.0,-0.669,0.0,0.024,0.0
I-art,-0.659,0.0,6.905,0.0,0.0,-1.144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.115,0.0,-0.735,0.0
B-eve,-0.696,0.0,0.0,0.0,7.473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.276,0.0
I-eve,-0.301,0.0,0.0,0.0,7.056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.763,0.0
B-geo,0.556,0.438,0.0,0.518,0.0,0.0,8.494,0.611,0.0,0.0,0.0,0.808,0.0,1.084,0.0,2.173,0.0
I-geo,-0.547,0.747,0.0,0.0,0.0,0.0,6.952,-1.17,0.0,0.0,0.0,-0.284,0.0,0.054,0.0,0.54,0.0
B-gpe,1.065,-1.02,0.0,-0.372,0.0,0.87,0.0,0.0,7.173,0.0,0.0,1.931,0.0,1.528,0.0,0.718,0.0
I-gpe,-0.43,0.0,0.0,0.0,0.0,-0.558,0.0,0.0,6.262,0.0,0.0,0.0,0.0,0.539,0.0,0.0,0.0
B-nat,-1.034,0.0,0.0,0.0,0.0,0.48,0.0,0.0,0.0,0.0,6.517,0.0,0.0,-0.017,0.0,-0.1,0.0
